## 1. Import Libraries

In [22]:
# Feature_Engineering
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import *
from pyspark.sql.functions import col, count, sum
# Modeling
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import linear_model, datasets
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score
import numpy as np

## 2. Load Data and Convert to Spark Data Frame

In [2]:
# read files
sc.textFile("loan.csv").take(1)

[u'id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m']

In [3]:
# load data as dataframe
loan_df=spark.read.csv("loan.csv",header=True)

In [4]:
loan_df.rdd.getNumPartitions()
type(loan_df)

pyspark.sql.dataframe.DataFrame

## 3. Create response variable and features
### 3.1 Remove some columns based on EDA results

In [64]:
loan_df1 = loan_df.drop('desc','mths_since_last_delinq','mths_since_last_record','next_pymnt_d',
                        'mths_since_last_major_derog','annual_inc_joint','dti_joint','verification_status_joint',
                        'open_acc_6m','open_il_6m','open_il_12m','open_il_24m','mths_since_rcnt_il','total_bal_il',
                        'il_util','open_rv_12m','open_rv_24m','max_bal_bc','all_util','inq_fi','total_cu_tl',
                        'inq_last_12m', # with a lot NA
                        'id','member_id','collection_recovery_fee','last_pymnt_amnt','last_pymnt_d','out_prncp','out_prncp_inv',
                        'pymnt_plan','recoveries','term','title','total_pymnt','total_pymnt_inv','total_rec_int',
                        'total_rec_late_fee','total_rec_prncp','url','verification_status', 'initial_list_status', 
                        'last_credit_pull_d','policy_code','emp_title','last_credit_pull_d' # domain knowledge
                       )

#is that intentionally not leaving out drops?

In [50]:
print loan_df1.head(1)
print len(loan_df1.columns)

[Row(loan_amnt=u'5000.0', funded_amnt=u'5000.0', funded_amnt_inv=u'4975.0', int_rate=u'10.65', installment=u'162.87', grade=u'B', sub_grade=u'B2', emp_length=u'10+ years', home_ownership=u'RENT', annual_inc=u'24000.0', issue_d=u'Dec-2011', loan_status=u'Fully Paid', purpose=u'credit_card', zip_code=u'860xx', addr_state=u'AZ', dti=u'27.65', delinq_2yrs=u'0.0', earliest_cr_line=u'Jan-1985', inq_last_6mths=u'1.0', open_acc=u'3.0', pub_rec=u'0.0', revol_bal=u'13648.0', revol_util=u'83.7', total_acc=u'9.0', collections_12_mths_ex_med=u'0.0', application_type=u'INDIVIDUAL', acc_now_delinq=u'0.0', tot_coll_amt=None, tot_cur_bal=None, total_rev_hi_lim=None)]
30


In [115]:
loan_df2=loan_df.select(
    loan_df.loan_amnt.cast("integer"),
    loan_df.funded_amnt.cast("integer"),
    loan_df.funded_amnt_inv.cast("integer"),
    loan_df.int_rate.cast("integer"),  #convert to float
    loan_df.installment.cast("integer"), #convert to float
    'grade',
    #'sub_grade',   # NEED TO BE DUMMIED
    #'emp_title',  ignore, (too many categories)
    'emp_length',  #make it dummy done
    'home_ownership', #group & make it dummy, done 
    loan_df.annual_inc.cast("integer"),
    # 'issue_d',  #why comment out?
    'loan_status', 
    # response variable (pending for change after group discussion)
    'purpose',  #make it dummy
    # 'zip_code', #interesting, worth dummying 
    # 'addr_state', #dummy
    loan_df.dti.cast("float"),#float done
    loan_df.delinq_2yrs.cast("integer"),  
    #take out NAs, 2? 1, 3, mostly 0

    # 'earliest_cr_line',
    loan_df.inq_last_6mths.cast("integer"), #similar to delingq_2yr
    loan_df.open_acc.cast("integer"), 
    loan_df.pub_rec.cast("integer"), #what is it? 0,1
    loan_df.revol_bal.cast("float"), #done
    loan_df.revol_util.cast("float"),#done
    loan_df.total_acc.cast("integer"),
    #loan_df.last_credit_pull_d.cast("integer"), #date
    'application_type', #dummy
    loan_df.acc_now_delinq.cast("integer"),#ok
    loan_df.tot_coll_amt.cast("float"), #done
    loan_df.tot_cur_bal.cast("float"), #done
    loan_df.total_rev_hi_lim.cast("integer") #what is this
)

### 3.2 Create response variable and remove rows with no valid response variable 

In [116]:
# needs to fix that, encoding doesn't seem to be right

def whetherpaid(x):
    if x in ['Default', 'Charged Off', 'Does not meet the credit policy. Status:Charged Off']:
        return 1
    elif x in ['Does not meet the credit policy. Status:Fully Paid', 'Fully Paid']:
        return 0
    else:
        return -1

In [117]:
paidflag = udf(lambda x: whetherpaid(x))

In [118]:
#subject to change
loan_df3 = loan_df2.withColumn(
    'paid_flag',
    paidflag('loan_status').cast("integer")
        ).where("paid_flag != -1").drop('loan_status')

In [119]:
loan_df3.printSchema()


root
 |-- loan_amnt: integer (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: integer (nullable = true)
 |-- int_rate: integer (nullable = true)
 |-- installment: integer (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: integer (nullable = true)
 |-- purpose: string (nullable = true)
 |-- dti: float (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- inq_last_6mths: integer (nullable = true)
 |-- open_acc: integer (nullable = true)
 |-- pub_rec: integer (nullable = true)
 |-- revol_bal: float (nullable = true)
 |-- revol_util: float (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- application_type: string (nullable = true)
 |-- acc_now_delinq: integer (nullable = true)
 |-- tot_coll_amt: float (nullable = true)
 |-- tot_cur_bal: float (nullable = true)
 |-- total_rev_hi_lim: integer (nullable = true)
 |-- paid_f

### 3.3 Create features
#### 3.3.1 Creating a category feature for "Loan Purpose"
There are 14 types of loan purpose. StringIndexer encodes a string column of labels to a column of label indices, and most frequent label gets index 0.

In [120]:
indexer = StringIndexer(inputCol="purpose", outputCol="purposeIndex")
loan_df4 = indexer.fit(loan_df3).transform(loan_df3).drop('purpose')

### 3.3.2 Create a numeric feature for "emp_length"

In [121]:
import re
def convert_to_int(s):
    s = re.sub('\\D', '', s)  #remove any non-digital character
    #\d matches any digital, #\D matches any non-digital
    try:
        return s
    except ValueError:
        return 'NaN'

emp_to_num = udf(convert_to_int)
loan_df5 = loan_df4.withColumn('emp_len',emp_to_num('emp_length')).drop('emp_length')
#not sure why above line doesn't seem to convert string to integer

In [122]:
loan_df5.emp_len.cast('integer')

Column<CAST(emp_len AS INT)>

### 3.3.3 Create numeric variable for grade 

In [123]:
# count how many nulls in each column
def count_null(c):
    return sum(col(c).isNull().cast("integer")).alias(c)

exprs = [count_null(c) for c in loan_df5.columns[0:9]]
loan_df5.agg(*exprs).show()

+---------+-----------+---------------+--------+-----------+-----+--------------+----------+---+
|loan_amnt|funded_amnt|funded_amnt_inv|int_rate|installment|grade|home_ownership|annual_inc|dti|
+---------+-----------+---------------+--------+-----------+-----+--------------+----------+---+
|        0|          0|              0|       0|          0|    0|             0|         4|252|
+---------+-----------+---------------+--------+-----------+-----+--------------+----------+---+



In [124]:
loan_df5.groupBy('grade').count().show()

+-----+-----+
|grade|count|
+-----+-----+
|    F| 7986|
|    E|19923|
|    B|76617|
|    D|41509|
|    C|66309|
|    A|42441|
|    G| 2154|
+-----+-----+



In [125]:
indexer = StringIndexer(inputCol="grade", outputCol="gradeIndex")
loan_df6 = indexer.fit(loan_df5).transform(loan_df5).drop('grade')

In [126]:
loan_df6.printSchema() #??

root
 |-- loan_amnt: integer (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: integer (nullable = true)
 |-- int_rate: integer (nullable = true)
 |-- installment: integer (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: integer (nullable = true)
 |-- dti: float (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- inq_last_6mths: integer (nullable = true)
 |-- open_acc: integer (nullable = true)
 |-- pub_rec: integer (nullable = true)
 |-- revol_bal: float (nullable = true)
 |-- revol_util: float (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- application_type: string (nullable = true)
 |-- acc_now_delinq: integer (nullable = true)
 |-- tot_coll_amt: float (nullable = true)
 |-- tot_cur_bal: float (nullable = true)
 |-- total_rev_hi_lim: integer (nullable = true)
 |-- paid_flag: integer (nullable = true)
 |-- purposeIndex: double (nullable = true)
 |-- emp_len: string (nullable = true)
 |--

### 3.3.4 Create numeric variable for grade 

In [127]:
loan_df6.groupBy('home_ownership').count().show()

+--------------+------+
|home_ownership| count|
+--------------+------+
|           OWN| 22282|
|          RENT|107831|
|      MORTGAGE|126598|
|           ANY|     1|
|         OTHER|   179|
|          NONE|    48|
+--------------+------+



In [128]:
def home_ownership_func(x):
    if x in ['ANY','OTHER','NONE']:
        return 'Other'
    else: 
        return x

home_ownership = udf(home_ownership_func)  #fixed a function
loan_df7 = loan_df6.withColumn('homeownership',home_ownership('home_ownership')).drop('home_ownership')

In [129]:
loan_df7.show(2)

+---------+-----------+---------------+--------+-----------+----------+-----+-----------+--------------+--------+-------+---------+----------+---------+----------------+--------------+------------+-----------+----------------+---------+------------+-------+----------+-------------+
|loan_amnt|funded_amnt|funded_amnt_inv|int_rate|installment|annual_inc|  dti|delinq_2yrs|inq_last_6mths|open_acc|pub_rec|revol_bal|revol_util|total_acc|application_type|acc_now_delinq|tot_coll_amt|tot_cur_bal|total_rev_hi_lim|paid_flag|purposeIndex|emp_len|gradeIndex|homeownership|
+---------+-----------+---------------+--------+-----------+----------+-----+-----------+--------------+--------+-------+---------+----------+---------+----------------+--------------+------------+-----------+----------------+---------+------------+-------+----------+-------------+
|     5000|       5000|           4975|      11|        163|     24000|27.65|          0|             1|       3|      0|  13648.0|      83.7|        9

In [130]:
indexer = StringIndexer(inputCol="homeownership", outputCol="homeownershipIndex")
loan_df8 = indexer.fit(loan_df7).transform(loan_df7).drop('homeownership')

In [131]:
loan_df8.show(2)

+---------+-----------+---------------+--------+-----------+----------+-----+-----------+--------------+--------+-------+---------+----------+---------+----------------+--------------+------------+-----------+----------------+---------+------------+-------+----------+------------------+
|loan_amnt|funded_amnt|funded_amnt_inv|int_rate|installment|annual_inc|  dti|delinq_2yrs|inq_last_6mths|open_acc|pub_rec|revol_bal|revol_util|total_acc|application_type|acc_now_delinq|tot_coll_amt|tot_cur_bal|total_rev_hi_lim|paid_flag|purposeIndex|emp_len|gradeIndex|homeownershipIndex|
+---------+-----------+---------------+--------+-----------+----------+-----+-----------+--------------+--------+-------+---------+----------+---------+----------------+--------------+------------+-----------+----------------+---------+------------+-------+----------+------------------+
|     5000|       5000|           4975|      11|        163|     24000|27.65|          0|             1|       3|      0|  13648.0|     

#### Replace missing values with 0

In [133]:
loan_df9 = loan_df8.fillna(
    0.0, ['annual_inc','dti']
        ).where('delinq_2yrs is not null')

In [136]:
exprs = [count_null(c) for c in loan_df9.columns[0:8]] 
#WHAT ABOUT THE REST? from 9 onward
loan_df9.agg(*exprs).show()

+---------+-----------+---------------+--------+-----------+----------+---+-----------+
|loan_amnt|funded_amnt|funded_amnt_inv|int_rate|installment|annual_inc|dti|delinq_2yrs|
+---------+-----------+---------------+--------+-----------+----------+---+-----------+
|        0|          0|              0|       0|          0|         0|  0|          0|
+---------+-----------+---------------+--------+-----------+----------+---+-----------+



#### Add loan_inc_ratio feature 

def calculate_ratio(a, b):
    try:
        return a/float(b)
    except TypeError:
        return 'NaN'
    except ZeroDivisionError:
        return 'NaN'

ratio = udf(calculate_ratio)

In [146]:
def calculate_ratio(a, b):
    try:
        return a/float(b)
    except TypeError:
        return 'NaN'
    except ZeroDivisionError:
        return 'NaN'

ratio = udf(calculate_ratio) #define a udf_function ratio

In [143]:
loan_df10 = loan_df9.withColumn(
    'loan_inc_ratio',ratio('loan_amnt','annual_inc'
                          ).cast('float'))

In [145]:
loan_df11 = loan_df10.withColumn(
    'instal_inc_ratio',
    ratio('installment','annual_inc').cast('float'))

### Add feature instal_inc_ratio


In [147]:
def calculate_monthly_ratio(a, b):
    try:
        return a/(float(b)/12)
    except TypeError:
        return 'NaN'
    except ZeroDivisionError:
        return 'NaN'
    
monthly_ratio = udf(calculate_monthly_ratio)

In [149]:
loan_df11 = loan_df10.withColumn(
    'instal_inc_ratio',
    ratio('installment','annual_inc').cast('float'))

In [150]:
loan_df11.printSchema()

root
 |-- loan_amnt: integer (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: integer (nullable = true)
 |-- int_rate: integer (nullable = true)
 |-- installment: integer (nullable = true)
 |-- annual_inc: integer (nullable = true)
 |-- dti: float (nullable = false)
 |-- delinq_2yrs: integer (nullable = true)
 |-- inq_last_6mths: integer (nullable = true)
 |-- open_acc: integer (nullable = true)
 |-- pub_rec: integer (nullable = true)
 |-- revol_bal: float (nullable = true)
 |-- revol_util: float (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- application_type: string (nullable = true)
 |-- acc_now_delinq: integer (nullable = true)
 |-- tot_coll_amt: float (nullable = true)
 |-- tot_cur_bal: float (nullable = true)
 |-- total_rev_hi_lim: integer (nullable = true)
 |-- paid_flag: integer (nullable = true)
 |-- purposeIndex: double (nullable = true)
 |-- emp_len: string (nullable = true)
 |-- gradeIndex: double (nullable = true)
 |-- ho